# Das automatisierte Krankenhaus

Wir wollen einen Klassifikator bauen, der anhand von gemessenen Daten entscheidet, ob ein Patient im Krankenhaus oder ambulant behandelt wird. 

Die Daten sind Teil der Veröffentlichung "Sadikin, Mujiono (2020), “EHR Dataset for Patient Treatment Classification”, Mendeley Data" und verfügbar unter: [Link] (https://data.mendeley.com/datasets/7kv3rctx7m/1/files/e1aea094-1201-4b28-a9ce-53dd8117f02c)

## 1. Dateien einlesen

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Hochladen Datensatz über eine Eingabemaske von lokalem Rechner etc.
from google.colab import files
files.upload()
Patients_df = pd.read_csv('data-ori.csv', sep=',')

Saving data-ori.csv to data-ori.csv


In [3]:
Patients_df

,HAEMATOCRIT,HAEMOGLOBINS,ERYTHROCYTE,LEUCOCYTE,THROMBOCYTE,MCH,MCHC,MCV,AGE,SEX,SOURCE
0,35.1,11.8,4.65,6.3,310,25.4,33.6,75.5,1,F,out
1,43.5,14.8,5.39,12.7,334,27.5,34.0,80.7,1,F,out
2,33.5,11.3,4.74,13.2,305,23.8,33.7,70.7,1,F,out
3,39.1,13.7,4.98,10.5,366,27.5,35.0,78.5,1,F,out
4,30.9,9.9,4.23,22.1,333,23.4,32.0,73.0,1,M,out
...,...,...,...,...,...,...,...,...,...,...,...
4407,32.8,10.4,3.49,8.1,72,29.8,31.7,94.0,92,F,in
4408,33.7,10.8,3.67,6.7,70,29.4,32.0,91.8,92,F,in
4409,33.2,11.2,3.47,7.2,235,32.3,33.7,95.7,93,F,out
4410,31.5,10.4,3.15,9.1,187,33.0,33.0,100.0,98,F,in


Hier eine kurze Erklärung der Daten, entnommen aus: [Link](https://www.kaggle.com/manishkc06/patient-treatment-classification)

* HAEMATOCRIT /Continuous /35.1 / Patient laboratory test result of haematocrit
* HAEMOGLOBINS/Continuous/11.8 / Patient laboratory test result of haemoglobins
* ERYTHROCYTE/Continuous/4.65 / Patient laboratory test result of erythrocyte
* LEUCOCYTE /Continuous /6.3 / Patient laboratory test result of leucocyte
* THROMBOCYTE/Continuous/310/ Patient laboratory test result of thrombocyte
* MCH/Continuous /25.4/ Patient laboratory test result of MCH
* MCHC/Continuous/33.6/ Patient laboratory test result of MCHC
* MCV/Continuous /75.5/ Patient laboratory test result of MCV
* AGE/Continuous/12/ Patient age
* SEX/Nominal – Binary/F/ Patient gender
* SOURCE/Nominal/ {1,0}/The class target 1.= in care patient, 0 = out care patient

## 2. Dateien aufbereiten

In [4]:
Patients_df.dtypes

HAEMATOCRIT     float64
HAEMOGLOBINS    float64
ERYTHROCYTE     float64
LEUCOCYTE       float64
THROMBOCYTE       int64
MCH             float64
MCHC            float64
MCV             float64
AGE               int64
SEX              object
SOURCE           object
dtype: object

Wir sehen daß wir noch 2 Spalten, SEX und SOURCE, in Zahlen umwandeln müssen (Machine Learning Algorithmen können nur damit arbeiten). 

Fangen wir mit Sex an und danach SOURCE, was ja auch das Label ist

In [5]:
print(Patients_df['SOURCE'].unique())
print(Patients_df['SEX'].unique())

['out' 'in']
['F' 'M']


Das Dataset ist fast direkt einsetzbar, nur müssen wir noch 2 Spalten bearbeiten: die letzte Spalte 'Source' in das Label umwandeln.

Wir nehmen folgende Werte für Source und damit das Label:
- 1 wenn der Patient im Krankenhaus bleibt
- 0 wenn der Patient ambulant behandelt wird

In [6]:
Sex_mapping = {"F": 1, "M": 0}

Patients_df['Sex']= Patients_df['SEX'].map(Sex_mapping) # baue eine neue Spalte Sex. Es wird für Sex der Wert 1 gesetzt, falls in der gleichen Zeile bei Source eine Fra steht.  
Patients_df['Sex'] = Patients_df['Sex'].astype('int')  # wandele die Spalte Target in den Datentyp Zahl um.
Patients_df = Patients_df.drop(['SEX'], axis=1) # jetzt brauchen wir die Spalte SEX nicht mehr
Patients_df.dtypes

HAEMATOCRIT     float64
HAEMOGLOBINS    float64
ERYTHROCYTE     float64
LEUCOCYTE       float64
THROMBOCYTE       int64
MCH             float64
MCHC            float64
MCV             float64
AGE               int64
SOURCE           object
Sex               int64
dtype: object

In [7]:
source_mapping = {"in": 1,
                 "out": 0}

Patients_df['Target']= Patients_df['SOURCE'].map(source_mapping) # baue eine neue Spalte Target. Es wird für Target der Wert 1 gesetzt, falls in der gleichen Zeile bei Source in steht.  
Patients_df['Target'] = Patients_df['Target'].astype('int')  # wandele die Spalte Target in den Datentyp Zahl um.
Patients_df = Patients_df.drop(['SOURCE'], axis=1) # jetzt brauchen wir die Spalte SOURCE nicht mehr
Patients_df.dtypes

HAEMATOCRIT     float64
HAEMOGLOBINS    float64
ERYTHROCYTE     float64
LEUCOCYTE       float64
THROMBOCYTE       int64
MCH             float64
MCHC            float64
MCV             float64
AGE               int64
Sex               int64
Target            int64
dtype: object

In [8]:
# Label/ Zielgröße in y abspeichern
y = Patients_df['Target']
X = Patients_df.iloc[:, :-1]  # verkürzte Schreibweise: alle Zeilen (der Doppelpunkt vor dem Komma) und zu den Zeilen die jeweiligen Spalten bis hin zur vorletzten werden in X gespeichert.

# letzter Test. 
print("X hat folgenden Shape", X.shape) # X sollte 150 Zeilen mit jeweils 4 Spalten/ Features haben
print("y-Vektor hat folgenden Shape", y.shape) #Y sollte 150 Zeilen mit 1 Wert, der Zielvariablen haben

X hat folgenden Shape (4412, 10)
y-Vektor hat folgenden Shape (4412,)


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Test ob Aufteilung funktioniert hat.
print("Shapes von X_train und X_test", X_train.shape, X_test.shape)
print("Shapes von y_train und y_test", y_train.shape, y_test.shape)

Shapes von X_train und X_test (3088, 10) (1324, 10)
Shapes von y_train und y_test (3088,) (1324,)


## 3. Machine Learning

In [10]:
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix

def Model_accuracy(predicted_y_values, real_y_values):
  print(confusion_matrix(real_y_values, predicted_y_values)) #Confusion Matrix
  print(classification_report(real_y_values, predicted_y_values)) # Print summary report
  print('accuracy is ',accuracy_score(predicted_y_values, real_y_values)) # Print accuracy score

In [11]:
from sklearn import svm

SVC_basis = svm.SVC()
SVC_basis.fit(X_train, y_train)
y_pred_SVM_basis = SVC_basis.predict(X_test)

In [12]:
Model_accuracy(y_pred_SVM_basis, y_test)

[[714  52]
 [351 207]]
              precision    recall  f1-score   support

           0       0.67      0.93      0.78       766
           1       0.80      0.37      0.51       558

    accuracy                           0.70      1324
   macro avg       0.73      0.65      0.64      1324
weighted avg       0.72      0.70      0.66      1324

accuracy is  0.695619335347432


Jetzt sind Sie an der Reihe!!! Probieren Sie die verschiedenen Verfahren aus der Vorlesung aus!

## Zum Schluß: das Modell im Einsatz

Wir testen jetzt wie das Modell im Einsatz wäre....und zwar an beliebingen Patienten

In [13]:
# Zur Wiederholung schnell noch die Datenstruktur von X_test. In dem Format müssen wir unsere Daten eingeben
print(X_test.columns)

Index(['HAEMATOCRIT', 'HAEMOGLOBINS', 'ERYTHROCYTE', 'LEUCOCYTE',
       'THROMBOCYTE', 'MCH', 'MCHC', 'MCV', 'AGE', 'Sex'],
      dtype='object')


HAEMATOCRIT /Continuous /35.1 / Patient laboratory test result of haematocrit
HAEMOGLOBINS/Continuous/11.8 / Patient laboratory test result of haemoglobins
ERYTHROCYTE/Continuous/4.65 / Patient laboratory test result of erythrocyte
LEUCOCYTE /Continuous /6.3 / Patient laboratory test result of leucocyte
THROMBOCYTE/Continuous/310/ Patient laboratory test result of thrombocyte
MCH/Continuous /25.4/ Patient laboratory test result of MCH
MCHC/Continuous/33.6/ Patient laboratory test result of MCHC
MCV/Continuous /75.5/ Patient laboratory test result of MCV
AGE/Continuous/12/ Patient age
SEX/Nominal – Binary/F/ Patient gender
SOURCE/Nominal/ {1,0}/The class target 1.= in care patient, 0 = out care patient


In [14]:
Patient = [[35.1, #HAEMATOCRIT
            11.8, #HAEMOGLOBINS
            4.65, # ERYTHROCYTE
            6.3, # LEUCOCYTE
            310, # THROMBOCYTE
            25.4, # MCH
            33.6, # MCHC
            75.5, # MCV
            12, # AGE
            1]] # Sexx

# make a prediction

if (SVC_basis.predict(Patient) == 0):
  print("Patient wird ambulant behandelt")
else:
  print("Patient bleibt stationär")

# alternativ geht auch folgende Schreibweise:
#print("Patient wird ambulant behandelt") if SVC_basis.predict(Patient) == 0 else print("Patient bleibt stationär")

Patient wird ambulant behandelt


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"
